In [1]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, \
    Dropout, Conv2DTranspose, Cropping2D, Add, UpSampling2D
from keras.layers.merge import concatenate
from keras_segmentation.models.model_utils import get_segmentation_model
from glob import glob

import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    input_height = 256
    input_width = 256
    n_classes = 3
    channels = 3

    img_input = Input(shape=(input_height,input_width, channels))

    conv0 = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
    conv0 = Dropout(0.2)(conv0)
    conv0 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv0)
    pool0 = MaxPooling2D((2, 2))(conv0)
    
    conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool0)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv3)

    up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)

    up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
    
    up3 = concatenate([UpSampling2D((2, 2))(conv5), conv0], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    out = Conv2D( n_classes, (1, 1) , padding='same')(conv6)

    model = get_segmentation_model(img_input ,  out ) # this would build the segmentation model

Using TensorFlow backend.


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


In [ ]:
model.train(
    train_images =  "/Users/mavaylon/Research/image-segmentation-keras/Data/train/img/",
    train_annotations = "/Users/mavaylon/Research/image-segmentation-keras/Data/train/ann/",
    epochs=20,
    steps_per_epoch=len(glob("/Users/mavaylon/Research/image-segmentation-keras/Data/train/img/*")),
    batch_size=1,
    validate=True,
    val_images="/Users/mavaylon/Research/image-segmentation-keras/Data/test/img/",
    val_annotations="/Users/mavaylon/Research/image-segmentation-keras/Data/test/ann/",
    val_batch_size=1,
    val_steps_per_epoch=len(glob("/Users/mavaylon/Research/image-segmentation-keras/Data/test/img/*"))
)

  0%|          | 0/5912 [00:00<?, ?it/s]

Verifying training dataset


  2%|▏         | 27/1478 [00:00<00:05, 265.01it/s]

Dataset verified! 
Verifying validation dataset


100%|██████████| 1478/1478 [00:05<00:00, 276.33it/s]


Dataset verified! 
hello_pet
Epoch 1/20
5912/5912 [==============================] - 8908s 2s/step - loss: 0.7929 - accuracy: 0.6518 - val_loss: 0.7328 - val_accuracy: 0.6816

Epoch 00001: saving model to pet_class_crf.h5
Epoch 2/20
5912/5912 [==============================] - 8887s 2s/step - loss: 0.7474 - accuracy: 0.6725 - val_loss: 0.7015 - val_accuracy: 0.7091

Epoch 00002: saving model to pet_class_crf.h5
Epoch 3/20
5912/5912 [==============================] - 8902s 2s/step - loss: 0.6973 - accuracy: 0.7006 - val_loss: 0.6278 - val_accuracy: 0.7223

Epoch 00003: saving model to pet_class_crf.h5
Epoch 4/20
5912/5912 [==============================] - 8895s 2s/step - loss: 0.6658 - accuracy: 0.7189 - val_loss: 0.6291 - val_accuracy: 0.7434

Epoch 00004: saving model to pet_class_crf.h5
Epoch 5/20
5912/5912 [==============================] - 8981s 2s/step - loss: 0.6473 - accuracy: 0.7280 - val_loss: 0.6390 - val_accuracy: 0.7514

Epoch 00005: saving model to pet_class_crf.h5
Epoch 